Make ROI Spheres from Coordinates
===
Author: Christopher Lin <clin5@bidmc.harvard.edu>

In [ ]:
from nilearn import datasets, image, plotting
from nimlab import datasets as nimds
import os
import numpy as np
import pandas as pd
from numpy.linalg import inv
from tqdm import tqdm
from glob import glob
%matplotlib inline

By default, this notebook assumes that the coordinates list is in MNI space. If it is already in voxel space, 
add `voxel_coord = True` to the call to `make_spheres` in the last cell 

Load source dataframe
---

In [ ]:
src_df = pd.read_csv('./my_coords.csv')
src_df.head()

Now, please specify which columns in the csv file to use as subject identifiers and coordinates in the dataframe:

In [ ]:
coord_df = pd.DataFrame({
    'subject_id': src_df['Experiment'],
    'mni_x': src_df['X'],
    'mni_y': src_df['Y'],
    'mni_z': src_df['Z'],
})

coord_df.head()

In [ ]:
def create_bin_sphere(arr_size, center, r):
    # https://stackoverflow.com/questions/53326570/how-to-create-sphere-inside-a-ndarray-python?noredirect=1&lq=1
    coords = np.ogrid[:arr_size[0], :arr_size[1], :arr_size[2]]
    distance = np.sqrt((coords[0] - center[0])**2 + (coords[1]-center[1])**2 + (coords[2]-center[2])**2) 
    return 1*(distance <= r)

def make_sphere(coord, brain_mask, radius, voxel_coord = False):
    #Transform from MNI coordinates to voxelwise(matrix) coords
    if voxel_coord == False:
        inv_affine = inv(brain_mask.affine)
        
        trans_raw_coord = image.coord_transform(coord[0], coord[1], coord[2], inv_affine)
        trans_coord = round(trans_raw_coord[0]), round(trans_raw_coord[1]), round(trans_raw_coord[2])

    else:
        trans_coord = coord

    bin_sphere = create_bin_sphere(brain_mask.shape, trans_coord, radius)
    sphere_img = image.new_img_like(brain_mask, bin_sphere)
    return sphere_img
    

Input required here:
---

In [ ]:
# Specify output directory:
outdir = './output_folder'
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
# Specify radius in voxels (NOT millimeters):
radius = 2

# Specify MNI brain mask to get affine transfrom from:
mni_brain = nimds.get_img("MNI152_T1_2mm_brain_mask")

for c in tqdm(coord_df.itertuples()):

    sphere = make_sphere((c.mni_x, c.mni_y, c.mni_z), mni_brain, radius)
    sphere.to_filename(outdir + '/' + c.subject_id + 
                       '_' + str(c.mni_x).replace('.','d') + 
                       '_' + str(c.mni_y).replace('.','d') + 
                       '_' + str(c.mni_z).replace('.','d') + 
                       '.nii.gz')

## Optional: concatenate spheres (e.g., for study-level maps)

In [ ]:
filenames = os.listdir(outdir)
groups = list(set(i.rsplit('_', 3)[0] for i in filenames))

In [ ]:
unique_groups = list(set(groups))
print(unique_groups)

In [ ]:
concat_dir = './merged_imgs'
if not os.path.exists(concat_dir):
    os.makedirs(concat_dir)

for i in unique_groups:
    !fsladd '{concat_dir}/ALL_{i}.nii.gz' `ls {outdir}/{i}*`

## Optional: visualize your spheres

In [ ]:
# OPTION ONE: visualize select spheres
plotting.plot_glass_brain('./concat_dir/filename')

In [ ]:
# OPTION TWO: visualize all your spheres (not recommended if you have a large number of files)
for file in os.listdir(concat_dir):
    plotting.plot_glass_brain(file, title=file)

In [ ]:
for file in os.listdir(concat_dir):
    plotting.plot_stat_map(file, title=file)